# Weather Predictions blender

In [1]:
# https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
from glob import glob
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

import shutil

# Project
project_common_path = os.path.dirname('.')
project_common_path = os.path.abspath(os.path.join(project_common_path, '..', 'common'))
if not project_common_path in sys.path:
    sys.path.append(project_common_path)

In [3]:
os.environ['THEANO_FLAGS'] = 'device=cpu'

import numpy as np
import pandas as pd

from data_utils import get_id_type_list_for_class, GENERATED_DATA, OUTPUT_PATH
from test_utils import create_submission

No test data found at D:\vfomin\MyExamples\python\planet_amazon_rainforest\common\..\input\test 
No additional test data found at D:\vfomin\MyExamples\python\planet_amazon_rainforest\common\..\input\test 


Using TensorFlow backend.


### Load predictions on trainval dataset

In [4]:
from data_utils import unique_tags, get_label

target_tags = ['target_' + t for t in unique_tags]
val_predictions_resnet_filepath = os.path.join(GENERATED_DATA, "val_predictions_ResNet50_2017-07-18-14-50.csv")
val_predictions_squeezenet_filepath = os.path.join(GENERATED_DATA, "val_predictions_SqueezeNet21_2017-07-13-22-23.csv")

val_predictions_vgg19_filepath = os.path.join(GENERATED_DATA, "val_predictions_deep_model_vgg19_train_LB092919_prob.csv")
val_predictions_weirdcnn_filepath = os.path.join(GENERATED_DATA, "val_predictions_deep_model_train_LB092655_prob.csv")

def get_val_predictions_df(val_predictions_filepath, search_prefix="val_predictions_*.csv"):
    if not os.path.exists(val_predictions_filepath):
        val_predictions_csv = glob(os.path.join(OUTPUT_PATH, search_prefix))
        df = pd.read_csv(val_predictions_csv[0]).dropna()
        for filepath in val_predictions_csv[1:]:
            df = pd.concat([df, pd.read_csv(filepath).dropna()])
        df.reset_index(inplace=True)   
        df.drop('index', axis=1, inplace=True)
        df['image_id'] = df['image_name'].apply(lambda x: int(x[len('train_'):]))    
        for t in target_tags:
            df[t] = ''
        def fill_target_tags(row):
            image_id = row[0]
            labels = get_label(image_id, "Train_jpg")
            row[1:] = labels    
            return row
        cols = ['image_id', ] + target_tags
        df[cols] = df[cols].apply(fill_target_tags, axis=1)

        df.to_csv(val_predictions_filepath, index=False)
        val_predictions_df = df
        df = None    
    else:
        val_predictions_df = pd.read_csv(val_predictions_filepath)
    return val_predictions_df


# val_predictions_resnet_df = get_val_predictions_df(val_predictions_resnet_filepath, "val_predictions_ResNet50*_2017-07-18-14-50.csv")
# val_predictions_squeezenet_df = get_val_predictions_df(val_predictions_squeezenet_filepath, "val_predictions_SqueezeNet21*_2017-07-18-14-50.csv")
val_predictions_vgg19_df = get_val_predictions_df(val_predictions_vgg19_filepath, "vgg19/deep_model_vgg19_train_LB092919_prob.csv")
val_predictions_weirdcnn_df = get_val_predictions_df(val_predictions_weirdcnn_filepath, "custom_weird_model/deep_model_train_LB092655_prob.csv")


In [7]:
print(len(val_predictions_resnet_df))
val_predictions_resnet_df.head()

40448


,image_name,agriculture,artisinal_mine,bare_ground,blooming,blow_down,clear,cloudy,conventional_mine,cultivation,...,target_conventional_mine,target_cultivation,target_habitation,target_haze,target_partly_cloudy,target_primary,target_road,target_selective_logging,target_slash_burn,target_water
0,train_18872,0.762381,0.007814,0.023866,0.002480,0.001474,0.983301,0.000370,0.002931,0.932843,...,0,1,1,0,0,1,0,0,1,1
1,train_10562,0.106635,0.000834,0.002179,0.001490,0.001146,0.000112,0.003351,0.001014,0.020308,...,0,0,0,0,1,1,0,0,0,0
2,train_28763,0.196588,0.004786,0.009777,0.002631,0.001869,0.019417,0.105915,0.004077,0.084854,...,0,1,1,0,1,1,0,0,0,0
3,train_31735,0.084816,0.001532,0.013537,0.010144,0.014883,0.633817,0.002344,0.002039,0.045785,...,0,0,0,0,0,1,0,0,0,0
4,train_320,0.106061,0.001101,0.004905,0.004541,0.001737,0.000195,0.000810,0.001513,0.102018,...,0,0,0,0,1,1,0,0,0,0


In [8]:
print(len(val_predictions_squeezenet_df))
val_predictions_squeezenet_df.head()

40320


,image_name,agriculture,artisinal_mine,bare_ground,blooming,blow_down,clear,cloudy,conventional_mine,cultivation,...,target_conventional_mine,target_cultivation,target_habitation,target_haze,target_partly_cloudy,target_primary,target_road,target_selective_logging,target_slash_burn,target_water
0,train_15438,0.986593,1.800301e-07,0.016957,3.479813e-08,2.728809e-07,0.986830,1.494415e-08,2.339067e-04,0.128825,...,0,1,0,1,0,1,1,0,0,0
1,train_12863,0.024612,7.483472e-08,0.000639,8.198895e-03,1.618286e-03,0.997336,1.701083e-08,9.840907e-09,0.007981,...,0,0,0,0,0,1,0,0,0,0
2,train_10519,0.721716,5.289071e-05,0.014284,1.069636e-04,1.935463e-03,0.000348,2.059214e-05,3.164162e-05,0.509124,...,0,0,0,0,1,1,0,0,0,0
3,train_17178,0.002469,9.790796e-11,0.000020,1.204151e-03,8.132902e-05,0.998747,8.051727e-10,1.947120e-11,0.000470,...,0,0,0,0,0,1,0,0,0,0
4,train_5177,0.066280,3.975656e-07,0.001354,5.240596e-04,2.015755e-04,0.348959,4.480021e-04,5.901127e-07,0.016081,...,0,0,0,0,0,1,0,0,0,0


In [14]:
len(val_predictions_resnet_df['image_name'].unique()), len(val_predictions_squeezenet_df['image_name'].unique()), 

(27158, 31357)

In [79]:
from metrics import score

def get_optimal_thresholds(y_true, y_preds):
    best_thresholds = [0.0]*len(unique_tags)    
    best_score = 0
    thrs = np.arange(0.0, 1.01, 0.01)    
    for i, tag in enumerate(unique_tags):
        print("%s : best_score=" % tag, end="")
        thresholds = list(best_thresholds)
        for thr in thrs:            
            thresholds[i] = thr
            s = score(y_true, y_preds > thresholds)
            if s > best_score:
                best_score = s
                best_thresholds[i] = thr
        print("%f, best_threshold=%f" % (best_score, best_thresholds[i]))
    return best_thresholds, best_score

## Train xgboost trees to make better predict weather

### Train to predict weather classes : 

In [33]:
weather_tags = ['clear', 'cloudy', 'haze', 'partly_cloudy']
weather_target_tags = ['target_%s' % t for t in weather_tags]
weather_labels = np.where(np.isin(unique_tags, weather_tags))[0]

In [34]:
weather_labels, [(i, t) for i, t in enumerate(unique_tags)], weather_target_tags

(array([ 5,  6, 10, 11], dtype=int64),
 [(0, 'agriculture'),
  (1, 'artisinal_mine'),
  (2, 'bare_ground'),
  (3, 'blooming'),
  (4, 'blow_down'),
  (5, 'clear'),
  (6, 'cloudy'),
  (7, 'conventional_mine'),
  (8, 'cultivation'),
  (9, 'habitation'),
  (10, 'haze'),
  (11, 'partly_cloudy'),
  (12, 'primary'),
  (13, 'road'),
  (14, 'selective_logging'),
  (15, 'slash_burn'),
  (16, 'water')],
 ['target_clear', 'target_cloudy', 'target_haze', 'target_partly_cloudy'])

In [35]:
val_predictions_df = val_predictions_vgg19_df

In [36]:
val_predictions_df.head()

,agriculture,artisinal_mine,bare_ground,blooming,blow_down,clear,cloudy,conventional_mine,cultivation,habitation,...,target_conventional_mine,target_cultivation,target_habitation,target_haze,target_partly_cloudy,target_primary,target_road,target_selective_logging,target_slash_burn,target_water
0,0.003752,1.428458e-09,0.000088,0.000230,0.000018,0.581344,2.108085e-05,2.150840e-09,0.001460,0.000500,...,0,0,0,1,0,1,0,0,0,0
1,0.799365,1.561381e-04,0.009740,0.000171,0.000109,0.998879,1.573375e-06,1.418633e-04,0.128822,0.038553,...,0,0,0,0,0,1,0,0,0,1
2,0.001674,1.083242e-09,0.000018,0.000292,0.000128,0.999015,1.269098e-06,8.769292e-11,0.000902,0.000416,...,0,0,0,0,0,1,0,0,0,0
3,0.005073,3.921103e-08,0.000086,0.006642,0.000504,0.998185,2.577778e-06,3.010564e-09,0.002385,0.002778,...,0,0,0,0,0,1,0,0,0,0
4,0.879332,1.331866e-03,0.017324,0.001006,0.000401,0.998223,4.311219e-07,3.768842e-04,0.343372,0.786686,...,0,0,1,0,0,1,1,0,0,0


In [10]:
import xgboost as xgb

d:\vfomin\myexamples\venv3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


#### A simple try of xgb

In [220]:
from sklearn.model_selection import StratifiedKFold

In [159]:
n_folds = 3

In [160]:
m = val_predictions_df[weather_target_tags].sum(axis=1) == 1
m2 = ~m

In [227]:

trainval_x = val_predictions_df[m][unique_tags].values
trainval_y = val_predictions_df[m][weather_target_tags].values

d = {
    (1, 0, 0, 0) : 0,
    (0, 1, 0, 0) : 1,    
    (0, 0, 1, 0) : 2,   
    (0, 0, 0, 1) : 3,    
}

def vector_to_index(trainval_y):
    
    output = np.zeros(len(trainval_y), dtype=np.uint8)
    for i, v in enumerate(trainval_y):        
        output[i] = d[tuple(v.tolist())]
    return output

trainval_y_ = vector_to_index(trainval_y)
    
skf = StratifiedKFold(n_splits=n_folds)
for train_index, test_index in skf.split(trainval_x, trainval_y_):
    train_x, val_x = trainval_x[train_index], trainval_x[test_index]
    train_y, val_y = trainval_y_[train_index], trainval_y_[test_index]
    
    print(train_x.shape, train_y.shape)
    print(train_x[:5, :], train_y[:5])
    print(val_x[:5, :], val_y[:5])
    break

(32380, 17) (32380,)
[[  6.04965031e-01   2.15695635e-03   7.51044676e-02   1.01612241e-03
    4.62560332e-04   1.75208211e-01   2.06295023e-04   7.55776477e-04
    5.43331623e-01   1.54359359e-02   8.81842554e-01   1.84541978e-02
    9.98271704e-01   3.75210531e-02   4.39244695e-03   4.38418332e-03
    2.68821627e-01]
 [  1.63640738e-01   8.02634804e-06   3.85102024e-03   1.35511495e-04
    1.37903524e-04   8.12684834e-01   2.27927212e-02   3.22563865e-05
    6.34065364e-03   7.66972499e-03   1.26849264e-01   1.25037991e-02
    9.31428313e-01   6.91066682e-01   3.30558891e-04   6.47022243e-05
    7.80763984e-01]
 [  4.39591426e-03   1.74956183e-09   7.79319234e-05   3.57810524e-04
    2.33775190e-05   6.20837331e-01   1.54425688e-05   2.05203743e-09
    1.48288393e-03   6.66052103e-04   4.02874887e-01   1.91877212e-03
    9.99889016e-01   3.08704283e-03   2.04650041e-05   5.34883384e-07
    4.17424552e-03]
 [  9.81193781e-02   5.00835449e-06   1.99962687e-03   2.17977031e-05
    2.905

In [230]:
params = {
    "objective": "multi:softprob",
    "booster": "gbtree",
    "eval_metric": "mlogloss",
    "learning_rate": 0.007,
    "tree_method": 'exact',
    "n_estimators": 150,
    "max_depth": 4,
    "subsample": 0.75,
    "colsample_bytree": 0.75,
    "silent": False,    
    "num_class": len(weather_target_tags)
}
num_boost_round = 2500
early_stopping_rounds = 100

In [231]:
dtrain = xgb.DMatrix(train_x, label=train_y)
dval = xgb.DMatrix(val_x, label=val_y)

In [232]:
watchlist = ((dtrain, 'train'), (dval, 'eval'))

In [233]:
gbm = xgb.train(params, dtrain, num_boost_round, evals=watchlist, early_stopping_rounds=early_stopping_rounds, verbose_eval=True)

[0]	train-mlogloss:1.37389	eval-mlogloss:1.37389
Multiple eval metrics have been passed: 'eval-mlogloss' will be used for early stopping.

Will train until eval-mlogloss hasn't improved in 100 rounds.
[1]	train-mlogloss:1.36161	eval-mlogloss:1.36162
[2]	train-mlogloss:1.34963	eval-mlogloss:1.34967
[3]	train-mlogloss:1.33782	eval-mlogloss:1.33786
[4]	train-mlogloss:1.32672	eval-mlogloss:1.32678
[5]	train-mlogloss:1.31574	eval-mlogloss:1.31581
[6]	train-mlogloss:1.30431	eval-mlogloss:1.30439
[7]	train-mlogloss:1.2931	eval-mlogloss:1.29319
[8]	train-mlogloss:1.28227	eval-mlogloss:1.28238
[9]	train-mlogloss:1.2713	eval-mlogloss:1.27142
[10]	train-mlogloss:1.26049	eval-mlogloss:1.26062
[11]	train-mlogloss:1.25066	eval-mlogloss:1.25079
[12]	train-mlogloss:1.24018	eval-mlogloss:1.24032
[13]	train-mlogloss:1.23111	eval-mlogloss:1.23128
[14]	train-mlogloss:1.22132	eval-mlogloss:1.22151
[15]	train-mlogloss:1.21117	eval-mlogloss:1.21136
[16]	train-mlogloss:1.20123	eval-mlogloss:1.20142
[17]	train

[156]	train-mlogloss:0.48374	eval-mlogloss:0.4861
[157]	train-mlogloss:0.481128	eval-mlogloss:0.483509
[158]	train-mlogloss:0.478535	eval-mlogloss:0.480933
[159]	train-mlogloss:0.475968	eval-mlogloss:0.478375
[160]	train-mlogloss:0.473506	eval-mlogloss:0.47593
[161]	train-mlogloss:0.471057	eval-mlogloss:0.473495
[162]	train-mlogloss:0.468567	eval-mlogloss:0.47102
[163]	train-mlogloss:0.466124	eval-mlogloss:0.468602
[164]	train-mlogloss:0.463656	eval-mlogloss:0.466147
[165]	train-mlogloss:0.461207	eval-mlogloss:0.463709
[166]	train-mlogloss:0.4588	eval-mlogloss:0.461317
[167]	train-mlogloss:0.456406	eval-mlogloss:0.45894
[168]	train-mlogloss:0.454013	eval-mlogloss:0.456569
[169]	train-mlogloss:0.451662	eval-mlogloss:0.454232
[170]	train-mlogloss:0.449329	eval-mlogloss:0.451922
[171]	train-mlogloss:0.446997	eval-mlogloss:0.449605
[172]	train-mlogloss:0.444773	eval-mlogloss:0.447396
[173]	train-mlogloss:0.442475	eval-mlogloss:0.445111
[174]	train-mlogloss:0.440242	eval-mlogloss:0.442883
[

[312]	train-mlogloss:0.250353	eval-mlogloss:0.255176
[313]	train-mlogloss:0.249531	eval-mlogloss:0.254374
[314]	train-mlogloss:0.24875	eval-mlogloss:0.253609
[315]	train-mlogloss:0.247944	eval-mlogloss:0.252815
[316]	train-mlogloss:0.24718	eval-mlogloss:0.252073
[317]	train-mlogloss:0.246428	eval-mlogloss:0.251342
[318]	train-mlogloss:0.245665	eval-mlogloss:0.250596
[319]	train-mlogloss:0.244878	eval-mlogloss:0.249828
[320]	train-mlogloss:0.244195	eval-mlogloss:0.249156
[321]	train-mlogloss:0.243435	eval-mlogloss:0.248413
[322]	train-mlogloss:0.242717	eval-mlogloss:0.247712
[323]	train-mlogloss:0.241984	eval-mlogloss:0.246989
[324]	train-mlogloss:0.241228	eval-mlogloss:0.246249
[325]	train-mlogloss:0.240474	eval-mlogloss:0.245505
[326]	train-mlogloss:0.239727	eval-mlogloss:0.244777
[327]	train-mlogloss:0.239	eval-mlogloss:0.244063
[328]	train-mlogloss:0.238314	eval-mlogloss:0.243393
[329]	train-mlogloss:0.237593	eval-mlogloss:0.242683
[330]	train-mlogloss:0.236875	eval-mlogloss:0.24198

[468]	train-mlogloss:0.174265	eval-mlogloss:0.181442
[469]	train-mlogloss:0.173978	eval-mlogloss:0.181179
[470]	train-mlogloss:0.173697	eval-mlogloss:0.180916
[471]	train-mlogloss:0.173428	eval-mlogloss:0.180654
[472]	train-mlogloss:0.173165	eval-mlogloss:0.180403
[473]	train-mlogloss:0.172917	eval-mlogloss:0.180167
[474]	train-mlogloss:0.172657	eval-mlogloss:0.17992
[475]	train-mlogloss:0.172379	eval-mlogloss:0.179662
[476]	train-mlogloss:0.172113	eval-mlogloss:0.179417
[477]	train-mlogloss:0.171849	eval-mlogloss:0.179168
[478]	train-mlogloss:0.171577	eval-mlogloss:0.178913
[479]	train-mlogloss:0.17136	eval-mlogloss:0.178715
[480]	train-mlogloss:0.171108	eval-mlogloss:0.178471
[481]	train-mlogloss:0.170855	eval-mlogloss:0.178229
[482]	train-mlogloss:0.170593	eval-mlogloss:0.177979
[483]	train-mlogloss:0.170343	eval-mlogloss:0.177742
[484]	train-mlogloss:0.170123	eval-mlogloss:0.177536
[485]	train-mlogloss:0.169879	eval-mlogloss:0.177304
[486]	train-mlogloss:0.169639	eval-mlogloss:0.17

[623]	train-mlogloss:0.146991	eval-mlogloss:0.156517
[624]	train-mlogloss:0.146883	eval-mlogloss:0.156429
[625]	train-mlogloss:0.146771	eval-mlogloss:0.156336
[626]	train-mlogloss:0.14666	eval-mlogloss:0.156237
[627]	train-mlogloss:0.146564	eval-mlogloss:0.15615
[628]	train-mlogloss:0.146454	eval-mlogloss:0.156057
[629]	train-mlogloss:0.146342	eval-mlogloss:0.155964
[630]	train-mlogloss:0.146247	eval-mlogloss:0.155885
[631]	train-mlogloss:0.146146	eval-mlogloss:0.155799
[632]	train-mlogloss:0.146046	eval-mlogloss:0.155713
[633]	train-mlogloss:0.145937	eval-mlogloss:0.15562
[634]	train-mlogloss:0.145844	eval-mlogloss:0.15554
[635]	train-mlogloss:0.145738	eval-mlogloss:0.155452
[636]	train-mlogloss:0.145636	eval-mlogloss:0.155369
[637]	train-mlogloss:0.145536	eval-mlogloss:0.155289
[638]	train-mlogloss:0.145441	eval-mlogloss:0.155214
[639]	train-mlogloss:0.145339	eval-mlogloss:0.15513
[640]	train-mlogloss:0.145245	eval-mlogloss:0.15505
[641]	train-mlogloss:0.145144	eval-mlogloss:0.154963

[779]	train-mlogloss:0.135481	eval-mlogloss:0.147566
[780]	train-mlogloss:0.135433	eval-mlogloss:0.147532
[781]	train-mlogloss:0.135383	eval-mlogloss:0.147495
[782]	train-mlogloss:0.135337	eval-mlogloss:0.147465
[783]	train-mlogloss:0.135286	eval-mlogloss:0.147426
[784]	train-mlogloss:0.135235	eval-mlogloss:0.147392
[785]	train-mlogloss:0.135186	eval-mlogloss:0.147355
[786]	train-mlogloss:0.135135	eval-mlogloss:0.147315
[787]	train-mlogloss:0.135086	eval-mlogloss:0.147282
[788]	train-mlogloss:0.135039	eval-mlogloss:0.147252
[789]	train-mlogloss:0.134992	eval-mlogloss:0.147218
[790]	train-mlogloss:0.134942	eval-mlogloss:0.147189
[791]	train-mlogloss:0.134888	eval-mlogloss:0.147153
[792]	train-mlogloss:0.134839	eval-mlogloss:0.147114
[793]	train-mlogloss:0.134792	eval-mlogloss:0.147091
[794]	train-mlogloss:0.134742	eval-mlogloss:0.147063
[795]	train-mlogloss:0.134701	eval-mlogloss:0.147039
[796]	train-mlogloss:0.134663	eval-mlogloss:0.147015
[797]	train-mlogloss:0.134616	eval-mlogloss:0.

[935]	train-mlogloss:0.129631	eval-mlogloss:0.144269
[936]	train-mlogloss:0.129606	eval-mlogloss:0.144258
[937]	train-mlogloss:0.129582	eval-mlogloss:0.144247
[938]	train-mlogloss:0.129552	eval-mlogloss:0.144237
[939]	train-mlogloss:0.129525	eval-mlogloss:0.144221
[940]	train-mlogloss:0.129496	eval-mlogloss:0.144205
[941]	train-mlogloss:0.12947	eval-mlogloss:0.144195
[942]	train-mlogloss:0.129443	eval-mlogloss:0.144182
[943]	train-mlogloss:0.129411	eval-mlogloss:0.144172
[944]	train-mlogloss:0.129387	eval-mlogloss:0.144158
[945]	train-mlogloss:0.12936	eval-mlogloss:0.144143
[946]	train-mlogloss:0.129334	eval-mlogloss:0.144135
[947]	train-mlogloss:0.129309	eval-mlogloss:0.144129
[948]	train-mlogloss:0.12928	eval-mlogloss:0.144115
[949]	train-mlogloss:0.129253	eval-mlogloss:0.144102
[950]	train-mlogloss:0.129219	eval-mlogloss:0.14409
[951]	train-mlogloss:0.129191	eval-mlogloss:0.144088
[952]	train-mlogloss:0.129162	eval-mlogloss:0.144074
[953]	train-mlogloss:0.129135	eval-mlogloss:0.1440

[1089]	train-mlogloss:0.125865	eval-mlogloss:0.14295
[1090]	train-mlogloss:0.125844	eval-mlogloss:0.142945
[1091]	train-mlogloss:0.125819	eval-mlogloss:0.14294
[1092]	train-mlogloss:0.125798	eval-mlogloss:0.14293
[1093]	train-mlogloss:0.125776	eval-mlogloss:0.142923
[1094]	train-mlogloss:0.125754	eval-mlogloss:0.142921
[1095]	train-mlogloss:0.125736	eval-mlogloss:0.142924
[1096]	train-mlogloss:0.125717	eval-mlogloss:0.142921
[1097]	train-mlogloss:0.125701	eval-mlogloss:0.142916
[1098]	train-mlogloss:0.125679	eval-mlogloss:0.142912
[1099]	train-mlogloss:0.125657	eval-mlogloss:0.142904
[1100]	train-mlogloss:0.125634	eval-mlogloss:0.142893
[1101]	train-mlogloss:0.125612	eval-mlogloss:0.142889
[1102]	train-mlogloss:0.125588	eval-mlogloss:0.142887
[1103]	train-mlogloss:0.125569	eval-mlogloss:0.142885
[1104]	train-mlogloss:0.125546	eval-mlogloss:0.14288
[1105]	train-mlogloss:0.125529	eval-mlogloss:0.142879
[1106]	train-mlogloss:0.125511	eval-mlogloss:0.142875
[1107]	train-mlogloss:0.125491	e

[1242]	train-mlogloss:0.122764	eval-mlogloss:0.142411
[1243]	train-mlogloss:0.122743	eval-mlogloss:0.142408
[1244]	train-mlogloss:0.122727	eval-mlogloss:0.142403
[1245]	train-mlogloss:0.122708	eval-mlogloss:0.142401
[1246]	train-mlogloss:0.122691	eval-mlogloss:0.142396
[1247]	train-mlogloss:0.12267	eval-mlogloss:0.142391
[1248]	train-mlogloss:0.12265	eval-mlogloss:0.14239
[1249]	train-mlogloss:0.122633	eval-mlogloss:0.142386
[1250]	train-mlogloss:0.122613	eval-mlogloss:0.142383
[1251]	train-mlogloss:0.122593	eval-mlogloss:0.142378
[1252]	train-mlogloss:0.122574	eval-mlogloss:0.14238
[1253]	train-mlogloss:0.122557	eval-mlogloss:0.142382
[1254]	train-mlogloss:0.122533	eval-mlogloss:0.142381
[1255]	train-mlogloss:0.122517	eval-mlogloss:0.142382
[1256]	train-mlogloss:0.122497	eval-mlogloss:0.142373
[1257]	train-mlogloss:0.122479	eval-mlogloss:0.142371
[1258]	train-mlogloss:0.122461	eval-mlogloss:0.142367
[1259]	train-mlogloss:0.122444	eval-mlogloss:0.14236
[1260]	train-mlogloss:0.122421	ev

[1395]	train-mlogloss:0.119889	eval-mlogloss:0.142254
[1396]	train-mlogloss:0.119868	eval-mlogloss:0.142257
[1397]	train-mlogloss:0.119852	eval-mlogloss:0.142257
[1398]	train-mlogloss:0.11983	eval-mlogloss:0.142257
[1399]	train-mlogloss:0.119813	eval-mlogloss:0.142257
[1400]	train-mlogloss:0.119792	eval-mlogloss:0.14226
[1401]	train-mlogloss:0.119773	eval-mlogloss:0.14226
[1402]	train-mlogloss:0.119755	eval-mlogloss:0.142265
[1403]	train-mlogloss:0.119741	eval-mlogloss:0.142264
[1404]	train-mlogloss:0.119725	eval-mlogloss:0.142268
[1405]	train-mlogloss:0.11971	eval-mlogloss:0.142266
[1406]	train-mlogloss:0.119689	eval-mlogloss:0.142266
[1407]	train-mlogloss:0.11967	eval-mlogloss:0.142269
[1408]	train-mlogloss:0.119652	eval-mlogloss:0.142267
[1409]	train-mlogloss:0.119635	eval-mlogloss:0.142265
[1410]	train-mlogloss:0.119621	eval-mlogloss:0.142262
[1411]	train-mlogloss:0.119601	eval-mlogloss:0.142262
[1412]	train-mlogloss:0.119587	eval-mlogloss:0.142264
[1413]	train-mlogloss:0.119574	ev

Compare performances :

Train

In [234]:
y_true = val_predictions_df[m][target_tags].values[train_index]
y_preds_before = val_predictions_df[m][unique_tags].values[train_index]
# get_optimal_thresholds(y_true, y_preds_before)

```
agriculture : best_score=0.492248, best_threshold=0.140000
artisinal_mine : best_score=0.510153, best_threshold=0.180000
bare_ground : best_score=0.527707, best_threshold=0.080000
blooming : best_score=0.547661, best_threshold=0.110000
blow_down : best_score=0.570279, best_threshold=0.160000
clear : best_score=0.575409, best_threshold=0.150000
cloudy : best_score=0.598786, best_threshold=0.090000
conventional_mine : best_score=0.626382, best_threshold=0.090000
cultivation : best_score=0.648190, best_threshold=0.180000
habitation : best_score=0.676076, best_threshold=0.170000
haze : best_score=0.707198, best_threshold=0.100000
partly_cloudy : best_score=0.738221, best_threshold=0.190000
primary : best_score=0.739737, best_threshold=0.210000
road : best_score=0.773801, best_threshold=0.180000
selective_logging : best_score=0.823794, best_threshold=0.100000
slash_burn : best_score=0.882304, best_threshold=0.140000
water : best_score=0.935892, best_threshold=0.170000

([0.14000000000000001,   0.17999999999999999,  0.080000000000000002,  0.11,  0.16,  0.14999999999999999,  0.089999999999999997, 0.089999999999999997,  0.17999999999999999,  0.17000000000000001,  0.10000000000000001,  0.19,  0.20999999999999999, 0.17999999999999999,  0.10000000000000001,  0.14000000000000001,  0.17000000000000001], 0.93589220897257264)
```

In [235]:
def check_predictions(y_true, y_preds_before, y_preds_after):
    
    thrs = np.array([0.14000000000000001,   0.17999999999999999,  0.080000000000000002,  0.11,  0.16,  0.14999999999999999,  0.089999999999999997, 0.089999999999999997,  0.17999999999999999,  0.17000000000000001,  0.10000000000000001,  0.19,  0.20999999999999999, 0.17999999999999999,  0.10000000000000001,  0.14000000000000001,  0.17000000000000001])
    print("No boosting. Mean train error (with best thresholds): ", np.mean(np.abs(y_true - (y_preds_before > thrs).astype(np.uint8))))
    print("No boosting. Mean weather only train error (with best thresholds): ", np.mean(np.abs(y_true[:, weather_labels] - (y_preds_before[:, weather_labels] > thrs[weather_labels]).astype(np.uint8))))
    
    print("With boosting. Mean weather only train error: ", np.mean(np.abs(y_true[:, weather_labels] - (y_preds_after[:, weather_labels] > 0.5).astype(np.uint8))))
    print("With boosting. Mean train error (with threshold = 0.5): ", np.mean(np.abs(y_true - (y_preds_after > 0.5).astype(np.uint8))))
    
    print(get_optimal_thresholds(y_true, y_preds_after))
    

In [236]:
y_preds_after = y_preds_before.copy()
y_preds_weather_after = gbm.predict(xgb.DMatrix(train_x), ntree_limit=gbm.best_iteration+1)
y_preds_after[:, weather_labels] = y_preds_weather_after

check_predictions(y_true, y_preds_before, y_preds_after)

No boosting. Mean train error (with best thresholds):  0.045698143371
No boosting. Mean weather only train error (with best thresholds):  0.045460160593
With boosting. Mean weather only train error:  0.0245521927116
With boosting. Mean train error (with threshold = 0.5):  0.0285524833775
agriculture : best_score=0.492220, best_threshold=0.140000
artisinal_mine : best_score=0.510126, best_threshold=0.180000
bare_ground : best_score=0.527628, best_threshold=0.110000
blooming : best_score=0.547611, best_threshold=0.110000
blow_down : best_score=0.570212, best_threshold=0.110000
clear : best_score=0.575589, best_threshold=0.100000
cloudy : best_score=0.599290, best_threshold=0.130000
conventional_mine : best_score=0.626900, best_threshold=0.090000
cultivation : best_score=0.648638, best_threshold=0.180000
habitation : best_score=0.676545, best_threshold=0.180000
haze : best_score=0.708313, best_threshold=0.160000
partly_cloudy : best_score=0.739784, best_threshold=0.210000
primary : best_s

Test

In [237]:
y_true = val_predictions_df[m][target_tags].values[test_index]
y_preds_before = val_predictions_df[m][unique_tags].values[test_index]

In [170]:
# get_optimal_thresholds(y_true, y_preds_before)

```
agriculture : best_score=0.491723, best_threshold=0.140000
artisinal_mine : best_score=0.509622, best_threshold=0.240000
bare_ground : best_score=0.527223, best_threshold=0.130000
blooming : best_score=0.547259, best_threshold=0.090000
blow_down : best_score=0.569750, best_threshold=0.060000
clear : best_score=0.575042, best_threshold=0.130000
cloudy : best_score=0.598413, best_threshold=0.070000
conventional_mine : best_score=0.626027, best_threshold=0.270000
cultivation : best_score=0.647533, best_threshold=0.210000
habitation : best_score=0.675459, best_threshold=0.140000
haze : best_score=0.706729, best_threshold=0.120000
partly_cloudy : best_score=0.737847, best_threshold=0.190000
primary : best_score=0.739535, best_threshold=0.170000
road : best_score=0.773389, best_threshold=0.220000
selective_logging : best_score=0.823530, best_threshold=0.170000
slash_burn : best_score=0.882076, best_threshold=0.200000
water : best_score=0.935354, best_threshold=0.210000

([0.14000000000000001,   0.23999999999999999,  0.13,  0.089999999999999997,  0.059999999999999998,  0.13,  0.070000000000000007, 0.27000000000000002,  0.20999999999999999,  0.14000000000000001,  0.12,  0.19,  0.17000000000000001,  0.22, 0.17000000000000001, 0.20000000000000001,  0.20999999999999999], 0.93535448764300033)
```

In [238]:
y_preds_after = y_preds_before.copy()
y_preds_weather_after = gbm.predict(xgb.DMatrix(val_x), ntree_limit=gbm.best_iteration+1)
y_preds_after[:, weather_labels] = y_preds_weather_after

check_predictions(y_true, y_preds_before, y_preds_after)

No boosting. Mean train error (with best thresholds):  0.0456975578574
No boosting. Mean weather only train error (with best thresholds):  0.0441467028896
With boosting. Mean weather only train error:  0.0286490491479
With boosting. Mean train error (with threshold = 0.5):  0.0293972367905
agriculture : best_score=0.491539, best_threshold=0.130000
artisinal_mine : best_score=0.509423, best_threshold=0.220000
bare_ground : best_score=0.527188, best_threshold=0.130000
blooming : best_score=0.547149, best_threshold=0.070000
blow_down : best_score=0.569624, best_threshold=0.060000
clear : best_score=0.574860, best_threshold=0.130000
cloudy : best_score=0.598185, best_threshold=0.030000
conventional_mine : best_score=0.625894, best_threshold=0.360000
cultivation : best_score=0.647580, best_threshold=0.210000
habitation : best_score=0.675544, best_threshold=0.140000
haze : best_score=0.706919, best_threshold=0.110000
partly_cloudy : best_score=0.737968, best_threshold=0.090000
primary : best

Run CV with a random search of optimal parameters

In [240]:
def generate_params(iter_num, seed=None):
    if iter_num > 0:
        if seed is not None:
            np.random.seed(seed=seed)
        for z in range(iter_num):
            eta = np.random.uniform(0.05, 0.001)
            max_depth = np.random.randint(2, 6)
            subsample = np.random.uniform(0.5, 0.95)
            colsample_bytree = np.random.uniform(0.5, 0.95)
            yield eta, max_depth, subsample, colsample_bytree
    else:
        eta = 0.05
        max_depth = 3
        subsample = 0.8204967474962096
        colsample_bytree = 0.7089159774987868
        yield eta, max_depth, subsample, colsample_bytree

In [241]:
m = val_predictions_df[weather_target_tags].sum(axis=1) == 1
trainval_x = val_predictions_df[m][unique_tags].values
trainval_y = val_predictions_df[m][weather_target_tags].values

d = {
    (1, 0, 0, 0) : 0,
    (0, 1, 0, 0) : 1,    
    (0, 0, 1, 0) : 2,   
    (0, 0, 0, 1) : 3,    
}

def vector_to_index(trainval_y):
    
    output = np.zeros(len(trainval_y), dtype=np.uint8)
    for i, v in enumerate(trainval_y):        
        output[i] = d[tuple(v.tolist())]
    return output

trainval_y_ = vector_to_index(trainval_y)


In [ ]:

eval_metric = 'mlogloss'

seed = 2017
n_folds = 5
dtrainval = xgb.DMatrix(trainval_x, label=trainval_y_, feature_names=unique_tags)

best_params = {
    'test-%s-mean' % eval_metric: 1e10,
    'params': None,
}
    
iter_num = 10
for (eta, max_depth, subsample, colsample_bytree) in list(generate_params(iter_num, seed=seed)):
    print('XGBoost params. ETA: {}, MAX_DEPTH: {}, SUBSAMPLE: {}, COLSAMPLE_BY_TREE: {}'.format(eta, max_depth,subsample,colsample_bytree))
    params = {
        "objective": "multi:softprob",
        "booster": "gbtree",
        "eval_metric": eval_metric,
        "eta": eta,
        "tree_method": 'exact',
        "max_depth": max_depth,
        "subsample": subsample,
        "colsample_bytree": colsample_bytree,
        "silent": 1,
        "seed": seed, 
        "num_class": len(weather_target_tags)
    }
    num_boost_round = 2500
    early_stopping_rounds = 100

    cvresult = xgb.cv(params, dtrain=dtrainval,
                       seed=params['seed'], stratified=True,
                       num_boost_round=num_boost_round, 
                       early_stopping_rounds=early_stopping_rounds, nfold=n_folds, verbose_eval=False)

    min_test_logloss_mean = cvresult['test-%s-mean' % params['eval_metric']].min()
    if best_params['test-%s-mean' % params['eval_metric']] > min_test_logloss_mean:
        best_params['test-%s-mean' % params['eval_metric']] = min_test_logloss_mean
        best_params['params'] = params
        print("Best cv result: ", cvresult.loc[cvresult.index[-1], :])
        print("Best params: ", params)
            
        

XGBoost params. ETA: 0.04897294895510025, MAX_DEPTH: 4, SUBSAMPLE: 0.8241212099907093, COLSAMPLE_BY_TREE: 0.8849532533753195
Best cv result:  test-mlogloss-mean     0.144753
test-mlogloss-std      0.004451
train-mlogloss-mean    0.120154
train-mlogloss-std     0.000932
Name: 184, dtype: float64
Best params:  {'objective': 'multi:softprob', 'booster': 'gbtree', 'eval_metric': 'mlogloss', 'eta': 0.04897294895510025, 'tree_method': 'exact', 'max_depth': 4, 'subsample': 0.8241212099907093, 'colsample_bytree': 0.8849532533753195, 'silent': 1, 'seed': 2017, 'num_class': 4}
XGBoost params. ETA: 0.01222624046647642, MAX_DEPTH: 5, SUBSAMPLE: 0.7922976846990782, COLSAMPLE_BY_TREE: 0.5633019770976554
XGBoost params. ETA: 0.03865187746016863, MAX_DEPTH: 2, SUBSAMPLE: 0.9309433282690102, COLSAMPLE_BY_TREE: 0.9139775309770741
Best cv result:  test-mlogloss-mean     0.143572
test-mlogloss-std      0.004617
train-mlogloss-mean    0.135080
train-mlogloss-std     0.001078
Name: 303, dtype: float64
Best 

In [190]:
best_params

{'params': {'booster': 'gbtree',
  'colsample_bytree': 0.9139775309770741,
  'eta': 0.03865187746016863,
  'eval_metric': 'mlogloss',
  'max_depth': 2,
  'num_class': 4,
  'objective': 'multi:softprob',
  'seed': 2017,
  'silent': 1,
  'subsample': 0.9309433282690102,
  'tree_method': 'exact'},
 'test-mlogloss-mean': 0.14361539999999998}

Train xgboost model on all data

In [207]:
sss = StratifiedShuffleSplit()

for train_index, test_index in sss.split(trainval_x, trainval_y_):
    train_x, val_x = trainval_x[train_index], trainval_x[test_index]
    train_y, val_y = trainval_y_[train_index], trainval_y_[test_index]
    break
    
dtrain = xgb.DMatrix(train_x, label=train_y)
dval = xgb.DMatrix(val_x, label=val_y)
watchlist = [(dtrain, 'train'), (dval, 'eval')]
params = best_params['params']
num_boost_round = 2500
early_stopping_rounds = 100

gbm = xgb.train(params, dtrain, num_boost_round, evals=watchlist, early_stopping_rounds=early_stopping_rounds, verbose_eval=False)                
print("Best score: ", gbm.best_score)
best_params['gbm'] = gbm

y_true = val_predictions_df[m][target_tags].values[test_index]
y_preds_before = val_predictions_df[m][unique_tags].values[test_index]
y_preds_after = y_preds_before.copy()
y_preds_weather_after = gbm.predict(xgb.DMatrix(val_x), ntree_limit=gbm.best_iteration+1)
y_preds_after[:, weather_labels] = y_preds_weather_after

print("With boosting. Mean weather only train error: ", np.mean(np.abs(y_true[:, weather_labels] - (y_preds_after[:, weather_labels] > 0.5).astype(np.uint8))))
print("With boosting. Mean train error (with threshold = 0.5): ", np.mean(np.abs(y_true - (y_preds_after > 0.5).astype(np.uint8))))
    
best_thresholds, best_score = get_optimal_thresholds(y_true, y_preds_after)
print(best_threholds, best_score)
best_params['best_threholds'] = best_threholds
best_params['best_score'] = best_score


Best score:  0.15059


In [191]:
import pickle 
now = datetime.now()

xgb_best_params_filepath = os.path.join(GENERATED_DATA, 'xgb_best_params_%s.pkl' % str(now.strftime("%Y-%m-%d-%H-%M")))

with open(xgb_best_params_filepath, 'wb') as handle:
    pickle.dump(best_params, handle, protocol=pickle.HIGHEST_PROTOCOL)    

Boost test predictions

In [39]:
predictions_csv = glob(os.path.join(OUTPUT_PATH, "predictions_*2017-07-13-19-21.csv"))

prediction_df = pd.read_csv(predictions_csv[0]).dropna()
for filepath in predictions_csv[1:]:
    prediction_df = pd.concat([prediction_df, pd.read_csv(filepath).dropna()])
prediction_df.reset_index(inplace=True)   
prediction_df.drop('index', axis=1, inplace=True)

In [40]:
prediction_df.head()

,image_name,agriculture,artisinal_mine,bare_ground,blooming,blow_down,clear,cloudy,conventional_mine,cultivation,habitation,haze,partly_cloudy,primary,road,selective_logging,slash_burn,water
0,test_40308,0.980499,5.343570e-06,0.012761,1.083201e-06,1.659149e-05,0.026706,6.184839e-06,1.213845e-04,0.247138,0.230338,0.002780,0.955976,0.994307,0.509100,6.428061e-05,5.902686e-03,0.049497
1,test_36168,0.008954,1.964079e-09,0.000126,4.806151e-04,5.747763e-05,0.990458,1.741330e-08,7.339324e-10,0.002185,0.001168,0.006540,0.000672,0.999985,0.002998,2.039154e-04,6.675383e-07,0.004990
2,test_6070,0.998530,1.298099e-10,0.004025,1.012986e-10,1.425290e-09,0.999737,2.468224e-13,2.351865e-06,0.095436,0.819567,0.000283,0.000036,0.947903,0.991666,2.751586e-07,2.056249e-04,0.032477
3,test_5483,0.003073,3.214428e-09,0.000053,3.451585e-02,1.510115e-03,0.997953,1.104697e-08,2.678857e-10,0.001081,0.000312,0.000302,0.001118,0.999984,0.001043,3.596786e-03,6.073845e-07,0.002270
4,test_5532,0.004007,6.949826e-08,0.000144,1.529595e-01,6.396817e-03,0.998306,3.806774e-08,3.966572e-09,0.001861,0.000672,0.000304,0.000979,0.999972,0.002848,2.929822e-02,2.036211e-06,0.003847


In [41]:
y_preds_init = prediction_df[unique_tags].values
y_preds = np.zeros_like(y_preds_init)

for tag_index in range(len(unique_tags)):
    
    print("\n----------------\n Tag index: {}".format(tag_index))
    dtest = xgb.DMatrix(y_preds_init)
    gbm = best_params_for_tag_index[tag_index]['gbm']
    y_preds_ = gbm.predict(dtest, ntree_limit=gbm.best_iteration+1)        
    y_preds[:, tag_index] = y_preds_



----------------
 Tag index: 0

----------------
 Tag index: 1

----------------
 Tag index: 2

----------------
 Tag index: 3

----------------
 Tag index: 4

----------------
 Tag index: 5

----------------
 Tag index: 6

----------------
 Tag index: 7

----------------
 Tag index: 8

----------------
 Tag index: 9

----------------
 Tag index: 10

----------------
 Tag index: 11

----------------
 Tag index: 12

----------------
 Tag index: 13

----------------
 Tag index: 14

----------------
 Tag index: 15

----------------
 Tag index: 16


In [42]:
df = prediction_df.copy()
df[unique_tags] = y_preds

In [43]:
df.head()

,image_name,agriculture,artisinal_mine,bare_ground,blooming,blow_down,clear,cloudy,conventional_mine,cultivation,habitation,haze,partly_cloudy,primary,road,selective_logging,slash_burn,water
0,test_40308,0.522231,0.45251,0.252149,0.389114,0.455467,0.161270,0.123053,0.455544,0.522468,0.544217,0.443230,0.866382,0.750729,0.503381,0.268825,0.499394,0.248423
1,test_36168,0.477120,0.45251,0.159650,0.389114,0.455467,0.807806,0.122661,0.455544,0.391684,0.410128,0.443108,0.136900,0.729628,0.494106,0.260546,0.451676,0.141758
2,test_6070,0.522231,0.45251,0.222007,0.389114,0.455467,0.830000,0.132560,0.455544,0.527279,0.583079,0.440586,0.190396,0.609088,0.505878,0.259275,0.451676,0.238310
3,test_5483,0.477120,0.45251,0.158208,0.533957,0.478718,0.837238,0.122661,0.455544,0.391450,0.412522,0.435470,0.134498,0.761620,0.494106,0.327320,0.451676,0.127105
4,test_5532,0.477120,0.45251,0.161170,0.601405,0.455475,0.837238,0.122661,0.455544,0.404547,0.414466,0.435470,0.135214,0.761620,0.494106,0.620745,0.451676,0.132539


In [44]:
def compute_mean(df):
    gb = df.groupby('image_name')
    df2 = gb.agg(np.mean).reset_index()
    return df2

In [45]:
mean_df = compute_mean(df)

In [46]:
len(mean_df), len(df)

(61191, 183573)

In [47]:
create_submission(mean_df, info="squeezenet21_blended_3_folds", thresholds=best_thresholds)